1. https://github.com/bashhike/video-action-recognition

In [ ]:
# temporal_stream_data

import numpy as np
import sys,os
import pickle
import optical_flow_prep as ofp
import gc


def stackOF(chunk,img_rows,img_cols,jobType):
	if jobType == 'train':
		pickleFile = '../dataset/temporal_train_data.pickle'
	else :
		pickleFile = '../dataset/temporal_test_data.pickle'
	with open(pickleFile,'rb') as f1:
		temporal_train_data=pickle.load(f1)

	X_train,Y_train=ofp.stackOpticalFlow(chunk,temporal_train_data,img_rows,img_cols)
	gc.collect()
	return (X_train,Y_train)

In [ ]:
import numpy as np
import sys,os
import pickle
import optical_flow_prep as ofp
import gc


def stackOF(chunk,img_rows,img_cols,jobType):
	if jobType == 'train':
		pickleFile = '../dataset/temporal_train_data.pickle'
	else :
		pickleFile = '../dataset/temporal_test_data.pickle'
	with open(pickleFile,'rb') as f1:
		temporal_train_data=pickle.load(f1)

	X_train,Y_train=ofp.stackOpticalFlow(chunk,temporal_train_data,img_rows,img_cols)
	gc.collect()
	return (X_train,Y_train)

2. https://github.com/wushidonguc/two-stream-action-recognition-keras

In [ ]:
# validation optical flows

import csv
import numpy as np
import os.path
import random
import threading
from keras.utils import to_categorical
import cv2

def get_stacked_opt_flows(self, row):
        output = []
        opt_flow_dir_x = os.path.join(self.opt_flow_path, 'u', row[2])
        opt_flow_dir_y = os.path.join(self.opt_flow_path, 'v', row[2])

        # spatial parameters
        # crop at center for validation
        left = int((self.original_image_shape[0] - self.image_shape[0]) * 0.5)
        top = int((self.original_image_shape[1] - self.image_shape[1]) * 0.5)
        right = left + self.image_shape[0]
        bottom = top + self.image_shape[1]
            
        # temporal parameters
        total_frames = len(os.listdir(opt_flow_dir_x))
        if total_frames - self.opt_flow_len + 1 < self.n_snip:
            loop = True
            strart_frame_window_len = 1
        else:
            loop = False
            start_frame_window_len = (total_frames - self.opt_flow_len + 1) // self.n_snip # starting frame selection window length

        # loop over snippets
        for i_snip in range(self.n_snip):
            if loop:
                start_frame = i_snip % (total_frames - self.opt_flow_len + 1) + 1
            else:
                start_frame = int(0.5 * start_frame_window_len + 0.5) + start_frame_window_len * i_snip
            # Get the optical flow stack
            frames = range(start_frame, start_frame + self.opt_flow_len) # selected optical flow frames
            opt_flow_stack = []
            # loop over frames
            for i_frame in frames:
                # horizontal components
                img = None # reset to be safe
                img = cv2.imread(opt_flow_dir_x + '/frame' + "%06d"%i_frame + '.jpg', 0)
                img = np.array(img)
                img = img - np.mean(img) # mean substraction
                img = img[top: bottom, left: right]
                img = img / 255. # normalize pixels 
                opt_flow_stack.append(img)
    
                # vertical components
                img2 = None # reset to be safe
                img2 = cv2.imread(opt_flow_dir_y + '/frame' + "%06d"%i_frame + '.jpg', 0)
                img2 = np.array(img2)
                img2 = img2 - np.mean(img2) # mean substraction
                img2 = img2[top: bottom, left: right]
                img2 = img2 / 255. # normalize pixels 
                opt_flow_stack.append(img2)

            opt_flow_stack = np.array(opt_flow_stack)
            opt_flow_stack = np.swapaxes(opt_flow_stack, 0, 2)

            output.append(opt_flow_stack)

        output = np.array(output)

        return output

In [ ]:
# training
'''
 def __init__(self, num_of_snip=1, opt_flow_len=10, image_shape=(224, 224), original_image_shape=(341, 256), class_limit=None):
        """Constructor.
        opt_flow_len = (int) the number of optical flow frames to consider
        class_limit = (int) number of classes to limit the data to.
            None = no limit.
        """
        self.opt_flow_len = opt_flow_len
        self.num_of_snip = num_of_snip
        self.class_limit = class_limit
        self.image_shape = image_shape
        self.original_image_shape = original_image_shape
        self.opt_flow_path = os.path.join('/data', 'opt_flow')
        
        saved_model = None
        class_limit = None  # int, can be 1-101 or None
        num_of_snip = 1 # number of chunks used for each video 
        opt_flow_len = 10 # number of optical flow frames used
        image_shape=(224, 224)
        load_to_memory = False  # pre-load the sequences into memory
        batch_size = 64
        nb_epoch = 2222
        name_str = None

        # Get the data.
        self.data_list = self.get_data_list()

        # Get the classes.
        self.classes = self.get_classes()

        # Now do some minor data cleaning
        self.data_list = self.clean_data_list()
'''


def stack_generator(self, batch_size, train_test, name_str="N/D"):
    """Return a generator of optical frame stacks that we can use to train on. There are
    a couple different things we can return:
    """
    # Get the right dataset for the generator.
    train, test = self.split_train_test()
    data_list = train if train_test == 'train' else test

    idx = 0

    print("\nCreating %s generator with %d samples.\n" % (train_test,
        len(data_list)))

    while 1:
        idx += 1
        print("Generator yielding batch No.%d" % idx)
        if(train_test == 'test'):
            print("Validating for job: %s" % name_str)
        X, y = [], []

        # Generate batch_size samples.
        for _ in range(batch_size):
            # Reset to be safe.
            stack = []

            # Get a random sample.
            row = random.choice(data_list)

            # Get the stacked optical flows from disk.
            stack = self.get_stacked_opt_flows(row, train_test)

            X.append(stack)
            y.append(self.get_class_one_hot(row[1]))

        X = np.array(X)
        y = np.array(y)
        y = np.squeeze(y)

        yield X, y

def get_stacked_opt_flows(self, row, train_test, crop='corner', val_aug='center'):
    # crop options for training: corner, random
    # augmentation options for testing: resize, center

    opt_flow_stack = []                                                 # Definisco stack degli optical flows
    opt_flow_dir_x = os.path.join(self.opt_flow_path, 'u', row[2])      # path optical flow x
    opt_flow_dir_y = os.path.join(self.opt_flow_path, 'v', row[2])      # path optical flow y

    # spatial parameters -> Data Augmentation
    if train_test == 'train':
        if crop == 'random':
            # crop at center and four corners randomly for training
            left, top = random.choice([[0, 0], [0, self.original_image_shape[1] - self.image_shape[1]], [self.original_image_shape[0] - self.image_shape[0], 0], [self.original_image_shape[0] - self.image_shape[0], self.original_image_shape[1] - self.image_shape[1]], [int((self.original_image_shape[0] - self.image_shape[0]) * 0.5), int((self.original_image_shape[1] - self.image_shape[1]) * 0.5)]])
        else:
            # random crop for training set
            left = int((self.original_image_shape[0] - self.image_shape[0]) * random.random())
            top = int((self.original_image_shape[1] - self.image_shape[1]) * random.random())
    else:
        # crop at center for validation
        left = int((self.original_image_shape[0] - self.image_shape[0]) * 0.5)
        top = int((self.original_image_shape[1] - self.image_shape[1]) * 0.5)
    right = left + self.image_shape[0]
    bottom = top + self.image_shape[1]

    # temporal parameters
    total_frames = len(os.listdir(opt_flow_dir_x))
    '''
    Definisco i parametri di stacking
    total_frames = numero totale di frame del video
    num_of_snip = 1 # number of chunks used for each video (1 di base)
    opt_flow_len = 10 # number of optical flow frames used
    
    '''
    win_len = (total_frames - self.opt_flow_len) // self.num_of_snip # starting frame selection window length  # selezione dei frame
    if train_test == 'train':
        start_frame = int(random.random() * win_len) + 1
    else:
        start_frame = int(0.5 * win_len) + 1
    frames = [] # selected optical flow frames
    for i in range(self.num_of_snip):
        frames += range(start_frame + self.opt_flow_len * i, start_frame + self.opt_flow_len * (i + 1))

    if train_test == 'train' and random.random() > 0.5: # flipping
        flip = True
    else:
        flip = False

    # loop over frames
    for i_frame in frames:

        # horizontal components
        img = None # reset to be safe
        img = cv2.imread(opt_flow_dir_x + '/frame' + "%06d"%(i_frame) + '.jpg', 0)
        print(opt_flow_dir_x + '/frame' + "%06d"%(i_frame) + '.jpg')
        img = np.array(img)
        # mean substraction 
        img = img - np.mean(img)
        if train_test == 'train' or val_aug == 'center':
            # crop
            img = img[left : right, top : bottom]
        else:
            #resize
            img = cv2.resize(img, self.image_shape)
        img = img / 255. # normalize pixels 
        if flip:
            img = -img
        opt_flow_stack.append(img)

        # vertical components
        img2 = None # reset to be safe
        img2 = cv2.imread(opt_flow_dir_y + '/frame' + "%06d"%(i_frame) + '.jpg', 0)
        # mean substraction 
        img2 = np.array(img2)
        img2 = np.swapaxes(img2, 0, 1)
        img2 = img2 - np.mean(img2)
        if train_test == 'train' or val_aug == 'center':
            # crop
            img2 = img2[left : right, top : bottom]
        else:
            #resize
            img2 = cv2.resize(img2, self.image_shape)
        img2 = img2 / 255. # normalize pixels 
        opt_flow_stack.append(img2)

    opt_flow_stack = np.array(opt_flow_stack)
    opt_flow_stack = np.swapaxes(opt_flow_stack, 0, 1)
    opt_flow_stack = np.swapaxes(opt_flow_stack, 1, 2)

    # random horizontal flip for training sets
    if flip:
        opt_flow_stack = np.flip(opt_flow_stack, 0)

    return opt_flow_stack
